### Read data and import necessary libraries and functions

In [ ]:
# Import necessary functionality
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, regexp_replace, udf, cos, radians, sin, sqrt, atan2, year, concat, substring, lit, avg, regexp_extract, expr, min, max, sum, count
from pyspark.sql.window import Window
from pyspark.sql.types import FloatType
import matplotlib.pyplot as plt
import seaborn as sns
import math
import os

# Create a SparkSession
spark = SparkSession.builder.appName("WindEnergy").getOrCreate()

# Read wind core sites data and select relevant columns and convert column namees to english
sites_df = spark.read.option("sep", ";").csv("data/metobs_wind_core_sites.csv", header=True, inferSchema=True).select(
    col("Id"),
    col("Namn").alias("Name"),
    col("Latitud"),
    col("Longitud"),
    col("Aktiv").alias("Active")
)

# Read wind turbine data
turbine_df = spark.read.option("sep", ",").csv("data/VBK_export_allman_prod - Vindkraftverk.csv", header=True, inferSchema=True).select(
    col("Verk-ID").alias("Turbine-ID"),
    col("Status"),
    col("Placering").alias("Placement"),
    col("N-Koordinat").alias("N-Coordinate"),
    col("E-Koordinat").alias("E-Coordinate"),
    col("Maxeffekt (MW)").alias("Maxeffect (MW)"),
    col("Elområde").alias("Bidding_Zone"),
    col("Uppfört").alias("Built")
)

sites_df.show(n=5)
turbine_df.show(n=5)


### Data cleaning

In [ ]:
turbine_df.select("Placement").distinct().show()
turbine_df.select("Bidding_Zone").distinct().show()

# Strip column values
turbine_df = turbine_df.withColumn("Bidding_Zone", regexp_replace(col("Bidding_Zone"), "\\s+", ""))
turbine_df = turbine_df.withColumn("Placement", regexp_replace(col("Placement"), "\\s+", ""))

turbine_df.select("Placement").distinct().show()
turbine_df.select("Bidding_Zone").distinct().show()

### Filter DataFrames

In [ ]:
# Filter out turbines:
#    not on land, 
#    not in production and 
#    not in Bidding Zone 1 (Luleå)
#    turbines with less than 1.01 max effect
turbine_df = turbine_df.filter((col("Status") == "Uppfört") & (col("Placement") == "Land") & (col("Bidding_Zone") == "Luleå") & (col("Maxeffekt (MW)")<1.01))

# Filter out non-active wind core sites
sites_df = sites_df.filter((col("Active") == "Ja"))

### Define helper functions

In [ ]:
# Function for transformation sweref99 tm to longitude and latitude coordinates
def sweref99_to_latlon(E, N):
    # Constants for SWEREF99 TM projection
    E0 = 500000  # False Easting in meters
    N0 = 0       # False Northing in meters
    F0 = 0.9996  # Scale factor at central meridian
    lo0 = math.radians(15)  # Central meridian in radians
    a = 6378137.0  # Semi-major axis of WGS 84 ellipsoid in meters
    la0 = 0  # Latitude of projection origin in radians

    # Convert
    lat = la0 + (N - N0) / (a * F0)
    long = lo0 + (E - E0) / (a * F0 * cos(lat))

    return lat, long

# Calculate distance between points (E, N) and (lat, long)
def distance(E, N, lat, long):

    # Convert E-kooridnat and N-kooridnat to (lat1, lon1)
    lat1, lon1 = sweref99_to_latlon(E, N)
    
    # Convert to radians
    lat2 = radians(lat)
    lon2 = radians(long)
    
    # Haversine formula
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * atan2(sqrt(a), sqrt(1-a))
    
    # Radius of the Earth in kilometers (mean value)
    radius_earth = 6371.0
    
    # Calculate the distance
    distance = radius_earth * c
    
    return distance

distance_udf = udf(distance, FloatType())

### Map turbine to closest wind core site

In [ ]:
mapping_df = turbine_df.crossJoin(sites_df)

# Calculate distance between each turbine and wind core site
mapping_df = mapping_df.withColumn("distance(km)", distance(mapping_df["E-Coordinate"], mapping_df["N-Coordinate"], mapping_df["Latitud"], mapping_df["Longitud"]))

# Find the distance to the closes wind core site for each turbine
min_distance_df = mapping_df.groupBy("Turbine-ID").agg(min(col("distance(km)")).alias("distance(km)"))

# Map each wind turbine to the closest wind core site
mapping_df = mapping_df.join(min_distance_df, ["Turbine-ID", "distance(km)"], "inner").select("Turbine-ID", "distance(km)", "Maxeffect (MW)", "Built", "Id", "Name")
print("Number of turbines", mapping_df.count())
mapping_df.show(n=5)

### Read wind speed data

In [ ]:
# List to store individual DataFrames
dataframes = []

# Loop through all files in the folder
for filename in os.listdir("data/weather_data_mean/"):
    if filename.endswith('.csv'):  # Assuming your files have a .csv extension
        file_path = os.path.join("data/weather_data_mean/", filename)

        # Extract id from filename
        parts = filename.split('_')
        id = parts[3]

        temp_df = spark.read.option("sep", ";").csv(file_path, header=True, inferSchema=True)

        # Select necessary fields and format date and time
        temp_df = temp_df.filter(year('Datum') >= 2020)
        temp_df = temp_df.withColumn('timestamp', concat(col('Datum'), substring(col('Tid (UTC)').cast('string'), 11, 100)))
        temp_df = temp_df.select(col("Vindhastighet").alias("wind_speed"), col("timestamp"))
        temp_df = temp_df.withColumn('Id', lit(id))
        
        dataframes.append(temp_df)

# Concatenate the DataFrames
wind_df = dataframes[0]  # Initialize with the first DataFrame
for df in dataframes[1:]:
    wind_df = wind_df.union(df)

wind_df.show(n=5)

### Merge wind speed data with wind turbine mapping data

In [ ]:
# Join wind measurement data and wind turbine data
data_df = mapping_df.join(wind_df, "Id", "inner")

# For each timestamp filter out windturbines that were built on a later date
data_df = data_df.filter(col("timestamp")>col("Built"))

# Group by wind core site and timestamp counting the number of turbines mapped to it
# (wind_speed will be the same for each row)
data_df = data_df.groupBy("Id", "timestamp").agg(
    count("*").alias("Turbines"),
    min("wind_speed").alias("wind_speed"),
)

# Group by timestamp calculating average windspeed across wind core sites
# weighted by the number of turbines mapped to each wind core site
data_df = data_df.groupBy("timestamp").agg(
     (sum(expr("wind_speed * Turbines")) / sum("Turbines")).alias("weighted_avg_wind_speed")
)

data_df.show(n=5)

### Read and format electricity data

In [ ]:
def rollingAverage3h(file_path):
    df = spark.read.csv(file_path, header=True, inferSchema=True).select("MTU", "Wind Onshore  - Actual Aggregated [MW]")
    
    # just display the end time of the time frame
    df_with_time = df.withColumn("End_Time", regexp_extract(df["MTU"], r"(\d{2}\.\d{2}\.\d{4} \d{2}:\d{2} \(UTC\))", 1))
    
    # 3h sliding window average to adhere to wind measurements
    windowSpec = Window.orderBy("End_Time").rowsBetween(-2, 0)

    df_with_avg = df_with_time.withColumn("Rolling Average 3h in MWh", avg("Wind Onshore  - Actual Aggregated [MW]").over(windowSpec))

    # Select relevant fields
    df_clean = df_with_avg.select(
        col("End_Time"), 
        col("Rolling Average 3h in MWh")
    )
    
    return df_clean

df_average2021 = rollingAverage3h("data/production/SE1Onshore2021UTC.csv")

df_average2022 = rollingAverage3h("data/production/SE1Onshore2022UTC.csv")

# Join years into a single dataframe
df_both_years = df_average2021.union(df_average2022)

# format data and time to adhere to wind measurements fromatting
df_both_years = df_both_years.withColumn("timestamp",
                   expr("concat(substring(End_Time, 7, 4),'-' ,substring(End_Time, 4, 2),'-', substring(End_Time, 1, 2), ' ', substring(End_Time, 12, 5), ':00')")
                   ).select("Rolling Average 3h in MWh", "timestamp")

df_both_years.show(n=5, truncate=False)

### Merge electricity and wind data

In [ ]:
data_df = df_both_years.join(other=data_df, how="inner", on="timestamp")
data_df.show(n=5)

### Perform Analysis

In [ ]:
correlation = data_df.corr("weighted_avg_wind_speed", "Rolling Average 3h in MWh", method="pearson")

print(f"Linear Correlation between wind and energy: {correlation}")

In [ ]:
# Sample 10% of the data for visualization
sample_df = data_df.select("weighted_avg_wind_speed", "Rolling Average 3h in MWh").sample(False, 0.1, seed=1)

# Convert the DataFrame to Pandas for plotting
pandas_df = sample_df.select("weighted_avg_wind_speed", "Rolling Average 3h in MWh").toPandas()

# Create a  regplot
sns.regplot(x=pandas_df['weighted_avg_wind_speed'], y=pandas_df['Rolling Average 3h in MWh'], line_kws={"color": "red"})
plt.xlabel('Wind')
plt.ylabel('Energy')
plt.title('Scatter Plot of Wind vs. Energy')
plt.show()